## SCikit-Learn


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv("../data/Advertising.csv")
df.head()

,Unnamed: 0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [3]:
df = pd.read_csv("../data/Advertising.csv", index_col=0)
df.head()

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [4]:
number_of_samples, number_of_features = df.shape[0], df.shape[1]-1 # -1 because sales is a label and not a feature

number_of_samples, number_of_features

(200, 3)

In [5]:
X, y = df.drop('Sales', axis="columns"), df['Sales']
X.head()

,TV,Radio,Newspaper
1,230.1,37.8,69.2
2,44.5,39.3,45.1
3,17.2,45.9,69.3
4,151.5,41.3,58.5
5,180.8,10.8,58.4


In [6]:
y.head()

1    22.1
2    10.4
3     9.3
4    18.5
5    12.9
Name: Sales, dtype: float64

## sklearn - steps


1. train | test split, sometimes train|val|test split
2. scaling sometimes required
    - min-max scaling
    - standardardisation
    - ...
    - scale the training data
    - scale test data to the training data --> avoiding data leakage
3. Fit algorithm to training data
4. Predict test data
5. Evaluate


---


## Train | Test split


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((140, 3), (60, 3), (140,), (60,))

---

## Feature Scaling

### Normalisation (min-max feature scaling)
- $X' = \frac{X-X_{\min}}{X_{max}-X_{min}}$

### Feature standardisation
- $X' = \frac{X-\mu}{\sigma}$

In [9]:
from sklearn.preprocessing import MinMaxScaler # for scaling

# Instantiate a scaler instance
scaler = MinMaxScaler()
scaler.fit(X_train) # Important- use this for training data

scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

print(f"{scaled_X_train.min()=}")
print(f"{scaled_X_train.max()=}")
print(f"{scaled_X_test.min()=}")
print(f"{scaled_X_test.max()=}")

# note scaled_X_test.min !=0, scaled_X_test.max !=1
# 0<=scaled_X_train <=1
# 0.005964214711729622 <= scaled_X_test <= 0.005964214711729622
 

scaled_X_train.min()=0.0
scaled_X_train.max()=1.0
scaled_X_test.min()=0.005964214711729622
scaled_X_test.max()=1.1302186878727631


In [10]:
scaled_X_train.shape, scaled_X_test.shape

((140, 3), (60, 3))

## Linear regression

### Ordinary least squares

In [12]:
from sklearn.linear_model import LinearRegression

model_OLS = LinearRegression()

model_OLS.fit(scaled_X_train, y_train)
print(f"Parameters {model_OLS.coef_}") # beta_1, beta_2, beta_3
print(f"Intercept {model_OLS.intercept_}") # beta0

Parameters [13.02832938  9.88465985  0.69237469]
Intercept 2.7418553248528124


## Stochastic gradient descent

In [16]:
from sklearn.linear_model import SGDRegressor

model_sgd = SGDRegressor(loss="squared_error", max_iter=10000)
model_sgd.fit(scaled_X_train, y_train)
print(f"Parameters {model_sgd.coef_}") # beta_1, beta_2, beta_3
print(f"Intercept {model_sgd.intercept_}") # beta0


Parameters [11.98164342  9.01950376  1.34632485]
Intercept [3.55884233]


## Manual prediction

In [21]:
test_sample_features = scaled_X_test[0].reshape(1,-1)
test_sample_features

array([[0.54988164, 0.63709677, 0.52286282]])

In [23]:
test_sample_features.shape

(1, 3)

In [24]:
X_test.iloc[0].to_numpy()

array([163.3,  31.6,  52.9])

In [25]:
test_sample_label = y_test.values[0]
test_sample_features, test_sample_label


(array([[0.54988164, 0.63709677, 0.52286282]]), 16.9)

In [27]:
model_OLS.predict(test_sample_features)[0]

16.56539629743484

In [28]:
X_test.iloc[0].to_numpy()

array([163.3,  31.6,  52.9])

## Evaluation

In [31]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 1. predict on test data 

y_pred_OLS = model_OLS.predict(scaled_X_test)
y_pred_SGD = model_sgd.predict(scaled_X_test)

y_pred_OLS[:5]

array([16.5653963 , 21.18822792, 21.55107058, 10.88923816, 22.20231988])

In [32]:
y_pred_SGD[:5]

array([16.59756799, 20.82594873, 21.11900457, 11.31552671, 21.4061522 ])

In [33]:
y_test[:5].values

array([16.9, 22.4, 21.4,  7.3, 24.7])

In [35]:
mae_OLS = mean_absolute_error(y_test, y_pred_OLS)
mae_SGD = mean_absolute_error(y_test, y_pred_SGD)

mse_OLS = mean_squared_error(y_test, y_pred_OLS)
mse_SGD = mean_squared_error(y_test, y_pred_SGD)

rmse_OLS = np.sqrt(mse_OLS)
rmse_SGD = np.sqrt(mse_SGD)


print(f"{mae_OLS=:.4f} \t\t {mse_OLS=:.4f} \t {rmse_OLS=:.4f}")
print(f"{mae_SGD=:.4f} \t\t {mse_SGD=:.4f} \t {rmse_SGD=:.4f}")

mae_OLS=1.5117 		 mse_OLS=3.7968 	 rmse_OLS=1.9485
mae_SGD=1.5224 		 mse_SGD=4.0857 	 rmse_SGD=2.0213
